In [2]:
import sys
from pathlib import Path

project_root = Path.cwd().parent   # one level up
sys.path.insert(0, str(project_root))
#Import the tools
from src.parse_atlas import parser
from src.calculations import physics_calcs, combinatorics
from src.im_calculator import im_calculator
import math, awkward as ak, numpy as np
import atlasopenmagic as atom
import matplotlib.pyplot as plt
import os
import uproot
import logging

from typing import Dict, List, Optional, Tuple


In [3]:
def _calculate_combination_invariant_mass(
    fs_events: ak.Array,
    combination: Dict[str, int],
    config: Dict,
    calculator: im_calculator.IMCalculator,
    logger: logging.Logger,
    final_state: str,
    worker_num: Optional[int] = None
) -> Tuple[Optional[ak.Array], Optional[str]]:
    """
    Calculate invariant mass for a single combination.
    
    Filters, slices, and calculates invariant mass for the given combination.
    Returns None if no valid events found.
    
    Args:
        fs_events: Events for the final state
        combination: Particle combination dictionary
        config: Configuration dictionary
        calculator: IMCalculator instance
        logger: Logger instance
        final_state: Final state string (for logging)
        
    Returns:
        Tuple of (invariant mass array or None, skip reason string or None)
    """
    # Logging is handled at higher level, just debug here
    logger.debug(f"Processing combination: {combination} for final state: {final_state}")
    
    # Filter events by exact particle counts
    # When is_exact_count=True, this also extracts only the specified particle types
    filtered_events = calculator.filter_by_particle_counts(
        events=fs_events,
        particle_counts=combination,
        is_exact_count=True  # Use exact count matching for combinations
    )
    
    if len(filtered_events) == 0:
        return None, 'no_events_after_filter'
    
    # Slice events by field (e.g., top N by pt)
    field_to_slice_by = config.get("field_to_slice_by", "pt")
    sliced_events = calculator.slice_by_field(
        events=filtered_events,
        particle_counts=combination,
        field_to_slice_by=field_to_slice_by
    )
    
    if len(sliced_events) == 0:
        return None, 'no_events_after_slice'
    
    # Calculate invariant mass
    inv_mass = calculator.calculate_invariant_mass(sliced_events)
    
    if not ak.any(inv_mass):
        return None, 'empty_inv_mass'
    
    return inv_mass, None

In [4]:
root_files_path = "/storage/agrp/netalev/data/root_files/"
max_len = 0
fs_max, fs_events_max = None, None
for root_file in os.listdir(root_files_path):
    file = parser.AtlasOpenParser.parse_root_file(root_files_path + root_file, batch_size=None)
    
    calculator = im_calculator.IMCalculator(file)
    for fs, fs_events in calculator.group_by_final_state():
        fs_max, fs_events_max = fs, fs_events
        break
    break

In [10]:
root_files_path = "/storage/agrp/netalev/data/root_files/"

for root_file in os.listdir(root_files_path):
    file = parser.AtlasOpenParser.parse_root_file(root_files_path + root_file, batch_size=None)
    
    calculator = im_calculator.IMCalculator(file)
    im_array = _calculate_combination_invariant_mass(
        fs_events_max, 
        {"Electrons": 1, "Muons": 1, "Jets": 1, "Photons": 1},
        {"field_to_slice_by": "pt", "is_exact_count": True, "is_particle_counts_range": False},
        calculator,
        logging.getLogger(),
        fs_max,
        None
    )

    print(im_array)
    break



(None, 'no_events_after_filter')


In [ ]:
inv_mass_path = "/storage/agrp/netalev/data/inv_masses/"

for im_array_npy in os.listdir(inv_mass_path):
    im_array = np.load(inv_mass_path + im_array_npy)
    print(im_array)
    
    n, bins, patches = plt.hist(im_array, 50)

    plt.xlabel('mass')
    plt.ylabel('count events')
    plt.title(im_array_npy)
    # plt.axis([40, 160, 0, 0.03])
    plt.grid(True)
    plt.show()